In [354]:
!pip install lyricsgenius
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from lyricsgenius import Genius

In [355]:
url_dave_30s = 'https://davesmusicdatabase.blogspot.com/2014/07/top-100-songs-of-1930-1939.html'

headers = {"User-Agent": 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

response = requests.get(url_dave_30, headers=headers)

response.status_code

200

In [356]:
soup = BeautifulSoup(response.content)

In [357]:
tags

[<p class="description"><span>DavesMusicDatabase.com is devoted to ranking, rating, and reviewing music of all genres and eras. The DMDB blog serves up album and song reviews, best-of lists, music history snapshots, and music-related essays.</span></p>,
 <p></p>,
 <p></p>,
 <p>
 These are the top 100 songs from the 1930s according to Dave’s Music Database. Rankings are figured by combining sales figures, chart data, radio airplay, video airplay, streaming figures, awards, and appearances on best-of lists. 
 
 </p>,
 <p><!--&#8212;Video: Over the Rainbow ----><iframe allowfullscreen="" class="youtube-player" frameborder="0" height="315" src="//www.youtube.com/embed/1HRa4X07jdE" title="YouTube video player" type="text/html" width="500"></iframe></p>,
 <p>
 
 1. <a href="http://davesmusicdatabase.blogspot.com/2011/09/judy-garland-charts-with-over-rainbow.html">Over the Rainbow</a>...Judy Garland (1939) <br/>
 2. <a href="http://davesmusicdatabase.blogspot.com/2011/12/fred-astaire-hit-1-wi

In [358]:
#Obtengo un set de los tags presentes en el tag 'p'
tags = soup.find_all(name="p")

#Separa en elementos de una lista las canciones.
songs_30s = ''.join(i.text for i in tags[5:15]).split('\n')
songs_30 = tags[5].text.split('\n')
#Elimina los espacios de los extremos de cada elemento de la lista si
#los hubiese y devuelve una lista nueva con cada elemento
list_songs_30s = [i.strip() for i in songs_30s]

#Elimina los elementos vacios de la lista
list_songs_30s = list(filter(None, list_songs_30s))
#songs_30s = songs.split('\n')

list_songs_30s

['1. Over the Rainbow...Judy Garland (1939)',
 '2. Night and Day...Fred Astaire with Leo Reisman (1932)',
 '3. In the Mood...Glenn Miller (1939)',
 '4. Cheek to Cheek...Fred Astaire with Leo Reisman (1935)',
 "5. Stormy Weather (Keeps Rainin' All the Time)...Ethel Waters (1933)",
 '6. Begin the Beguine...Artie Shaw (1938)',
 '7. The Way You Look Tonight...Fred Astaire with Johnny Green (1936)',
 '8. Pennies from Heaven...Bing Crosby with George Stoll (1936)',
 '9. A-Tisket, A-Tasket...Ella Fitzgerald with Chick Webb (1938)',
 '10. All the Things You Are...Tommy Dorsey with Jack Leonard (1939)',
 '11. Silent Night...Bing Crosby (1935)',
 '12. Smoke Gets in Your Eyes…Paul Whiteman with Bob Lawrence (1933)',
 '13. Mood Indigo...Duke Ellington (1931)',
 '14. I Got Rhythm...Red Nichols (1930)',
 '15. Deep Purple...Larry Clinton with Bea Wain (1939)',
 '16. Happy Days Are Here Again...Ben Selvin (1930)',
 '17. God Bless America...Kate Smith (1939)',
 '18. Brother, Can You Spare a Dime?...Rud

In [359]:
def list_songs_url(url):
    headers = {"User-Agent": 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content)
    tags = soup.find_all(name="p")
    
    #Separa en elementos de una lista las canciones.
    songs = ''.join(i.text for i in tags[5:15]).split('\n')
    
    #Elimina los espacios de los extremos de cada elemento de la lista si
    #los hubiese y devuelve una lista nueva con cada elemento
    songs = [i.strip() for i in songs]
    
    #Elimina los elementos vacios de la lista
    songs = list(filter(None, songs))
    return songs

list_songs_url(url_dave)

['1. Over the Rainbow...Judy Garland (1939)',
 '2. Night and Day...Fred Astaire with Leo Reisman (1932)',
 '3. In the Mood...Glenn Miller (1939)',
 '4. Cheek to Cheek...Fred Astaire with Leo Reisman (1935)',
 "5. Stormy Weather (Keeps Rainin' All the Time)...Ethel Waters (1933)",
 '6. Begin the Beguine...Artie Shaw (1938)',
 '7. The Way You Look Tonight...Fred Astaire with Johnny Green (1936)',
 '8. Pennies from Heaven...Bing Crosby with George Stoll (1936)',
 '9. A-Tisket, A-Tasket...Ella Fitzgerald with Chick Webb (1938)',
 '10. All the Things You Are...Tommy Dorsey with Jack Leonard (1939)',
 '11. Silent Night...Bing Crosby (1935)',
 '12. Smoke Gets in Your Eyes…Paul Whiteman with Bob Lawrence (1933)',
 '13. Mood Indigo...Duke Ellington (1931)',
 '14. I Got Rhythm...Red Nichols (1930)',
 '15. Deep Purple...Larry Clinton with Bea Wain (1939)',
 '16. Happy Days Are Here Again...Ben Selvin (1930)',
 '17. God Bless America...Kate Smith (1939)',
 '18. Brother, Can You Spare a Dime?...Rud

In [360]:
test_str = '81. The Peanut Vendor (El Manisero)...Don Azpiazu with Antonio Machin (1930)'

#Recibe un string y devuelve el a;o de el string de la cancion
def extract_year(song_string):
    return song_string[-5:-1]

extract_year(test_str)

'1930'

In [361]:
#Recibe un string y devuelve el a;o de el string de la cancion
def extract_decade(song_string):
    return extract_year(song_string)[:3] + '0'

extract_decade(test_str)

'1930'

In [362]:
#Recibe un string y devuelve el orden de la cancion
def extract_order(song_string):
    return song_string[:3].replace('.','').strip()

extract_order(test_str)

'81'

In [363]:
#Recibe un string y devuelve el titulo de la cancion
def extract_title(song_string):
    return re.findall('\d+.\s(.*)(?:\.\.\.|\…)',song_string)[0]

extract_title(test_str)

'The Peanut Vendor (El Manisero)'

In [364]:
#Recibe un string y devuelve el autor de la cancion
def extract_author(song_string):
    return re.findall('(?:\.\.\.|\…)(.*)\(',song_string)[0].split('with')[0].strip()

extract_author(test_str)

'Don Azpiazu'

In [365]:
def ext_data_songs(url_dave):
    list_songs_dave = list_songs_url(url_dave)
    formated_songs = []
    for i in list_songs_dave:
        decade = extract_decade(i)
        year = extract_year(i)
        title = extract_title(i)
        author = extract_author(i)
        order = extract_order(i)
        
        song = {
            'decade':decade,
            'year':year,
            'title':title,
            'author':author,
            'order':order,
        }
        formated_songs.append(song)
    return formated_songs

ext_data_songs(url_dave_30)

[{'decade': '1930',
  'year': '1939',
  'title': 'Over the Rainbow',
  'author': 'Judy Garland',
  'order': '1'},
 {'decade': '1930',
  'year': '1932',
  'title': 'Night and Day',
  'author': 'Fred Astaire',
  'order': '2'},
 {'decade': '1930',
  'year': '1939',
  'title': 'In the Mood',
  'author': 'Glenn Miller',
  'order': '3'},
 {'decade': '1930',
  'year': '1935',
  'title': 'Cheek to Cheek',
  'author': 'Fred Astaire',
  'order': '4'},
 {'decade': '1930',
  'year': '1933',
  'title': "Stormy Weather (Keeps Rainin' All the Time)",
  'author': 'Ethel Waters',
  'order': '5'},
 {'decade': '1930',
  'year': '1938',
  'title': 'Begin the Beguine',
  'author': 'Artie Shaw',
  'order': '6'},
 {'decade': '1930',
  'year': '1936',
  'title': 'The Way You Look Tonight',
  'author': 'Fred Astaire',
  'order': '7'},
 {'decade': '1930',
  'year': '1936',
  'title': 'Pennies from Heaven',
  'author': 'Bing Crosby',
  'order': '8'},
 {'decade': '1930',
  'year': '1938',
  'title': 'A-Tisket, A-

In [366]:
dicc_30s = ext_data_songs(url_dave_30)
df_30s = pd.DataFrame(dicc_30s)
df_30s

,decade,year,title,author,order
0,1930,1939,Over the Rainbow,Judy Garland,1
1,1930,1932,Night and Day,Fred Astaire,2
2,1930,1939,In the Mood,Glenn Miller,3
3,1930,1935,Cheek to Cheek,Fred Astaire,4
4,1930,1933,Stormy Weather (Keeps Rainin' All the Time),Ethel Waters,5
...,...,...,...,...,...
95,1930,1937,Too Marvelous for Words,Bing Crosby,96
96,1930,1932,"Brother, Can You Spare a Dime?",Bing Crosby,97
97,1930,1930,Exactly Like You,Ruth Etting,98
98,1930,1934,The Carioca,Enric Madriguera,99


# Spotify

In [367]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="00e41fa4725d4469b29d9e36f43d7ec9",
                                                           client_secret="e64b15d24085495283f18937da5452d3"))

In [368]:
def search_spoti(title, author):
    search = sp.search(title + ' ' + author, market='US')
    tracks = search['tracks']
    df = pd.json_normalize(tracks['items'])
    df = df[['id', 'name', 'popularity', 'artists', 'album.name', 'external_urls.spotify', 'uri', 'explicit', 'album.release_date', 'duration_ms']]
    df['artists'] = df.artists.apply(lambda d: d[0]['name'])
    return df.head(1)

In [369]:
def songs_to_spoti_df(songs_df):
    lista = []

    for i in range(len(songs_df)):
        try:
            search = search_spoti(songs_df.loc[i, "title"], songs_df.loc[i, "author"])
            lista.append(search)
        except KeyError:
            pass
    
    return pd.concat(lista, axis=0)

songs_to_spoti_df(df_30s)

,id,name,popularity,artists,album.name,external_urls.spotify,uri,explicit,album.release_date,duration_ms
0,568SEFtDjKr7N2PytpA6D5,Over The Rainbow,49,Judy Garland,The Wizard of Oz (Original Motion Picture Soun...,https://open.spotify.com/track/568SEFtDjKr7N2P...,spotify:track:568SEFtDjKr7N2PytpA6D5,False,1939-08-25,165240
0,2b9FWk7vi5DpQypM3aSS9d,Night And Day,39,Fred Astaire,Mr. Top Hat,https://open.spotify.com/track/2b9FWk7vi5DpQyp...,spotify:track:2b9FWk7vi5DpQypM3aSS9d,False,1957-02-01,290840
0,1xsY8IFXUrxeet1Fcmk4oC,In the Mood,53,Glenn Miller,Ultimate Big Band Collection: Glenn Miller,https://open.spotify.com/track/1xsY8IFXUrxeet1...,spotify:track:1xsY8IFXUrxeet1Fcmk4oC,False,1939,216746
0,5qWXXF3On2xUfVu1ND1NLN,Cheek to Cheek,51,Fred Astaire,The Essential Fred Astaire,https://open.spotify.com/track/5qWXXF3On2xUfVu...,spotify:track:5qWXXF3On2xUfVu1ND1NLN,False,1935,198106
0,6O89uMM5FDGGRJ6aYLG51B,Stormy Weather (Keeps Rainin' All The Time) - ...,16,Ethel Waters,The Incomparable Ethel Waters,https://open.spotify.com/track/6O89uMM5FDGGRJ6...,spotify:track:6O89uMM5FDGGRJ6aYLG51B,False,1925,185840
...,...,...,...,...,...,...,...,...,...,...
0,47c6hVC9hL8KoAclJCVOXC,Marie,26,Tommy Dorsey,The Sentimental Gentleman Of Swing - The Tommy...,https://open.spotify.com/track/47c6hVC9hL8KoAc...,spotify:track:47c6hVC9hL8KoAclJCVOXC,False,1925,197786
0,5gUHFj66K17BiZaCiv4yS1,Too Marvelous For Words - Single Version,7,Bing Crosby,Bing-His Legendary Years 1931-1957,https://open.spotify.com/track/5gUHFj66K17BiZa...,spotify:track:5gUHFj66K17BiZaCiv4yS1,False,1993-01-01,186000
0,6DaE48egpIqB7qTgUHxSG8,"Brother, Can You Spare a Dime? (with Lenny Hay...",30,Bing Crosby,The Essential Bing Crosby (The Columbia Years),https://open.spotify.com/track/6DaE48egpIqB7qT...,spotify:track:6DaE48egpIqB7qTgUHxSG8,False,1932,192640
0,5e6AyGzxGycwpNaPY5To7v,Exactly Like You,0,Ruth Etting,Anthology Vol. 1,https://open.spotify.com/track/5e6AyGzxGycwpNa...,spotify:track:5e6AyGzxGycwpNaPY5To7v,False,2018-01-24,188017


# Genius

In [370]:
!pip install lyricsgenius

In [371]:
import lyricsgenius
genius = lyricsgenius.Genius(
    'v3s3cdBYSN4Kta3F2HDTmdqqNOckKEZTQA5O5Bhb5J-BV_L02jqPGCEnvpKsSrBz')

In [372]:
artist = genius.search_artist("Judy Garland", max_songs=3, sort="title")

Searching for songs by Judy Garland...

Song 1: "A Cottage For Sale"
Song 2: "A Foggy Day"
Song 3: "A Foggy Day (In London Town)"

Reached user-specified song limit (3).
Done. Found 3 songs.


In [373]:
song = artist.song("Over the Rainbow")

Searching for "Over the Rainbow" by Judy Garland...
Done.


In [374]:
song.lyrics

"Over the Rainbow Lyrics[Verse 1]\nSomewhere over the rainbow\nWay up high\nThere's a land that I heard of\nOnce in a lullaby\n\n[Verse 2]\nSomewhere over the rainbow\nSkies are blue\nAnd the dreams that you dare to dream\nReally do come true\n[Chorus]\nSomeday I'll wish upon a star\nAnd wake up where the clouds are far\nBehind me\nWhere troubles melt like lemon drops\nAway above the chimney tops\nThat's where you'll find me\n\n[Verse 3]\nSomewhere over the rainbow\nBluebirds fly\nBirds fly over the rainbow\nWhy, then, oh, why can't I?\n\n[Outro]\nIf happy little bluebirds fly\nBeyond the rainbow\nWhy, oh, why can't I?17Embed"

In [375]:
def get_lyrics(artist_name, song_name):
    genius = lyricsgenius.Genius(
    'v3s3cdBYSN4Kta3F2HDTmdqqNOckKEZTQA5O5Bhb5J-BV_L02jqPGCEnvpKsSrBz')
    artist = genius.search_artist(artist_name, max_songs=0, sort="title")
    song = artist.song(song_name)
    return re.sub("[\(\[].*?[\)\]]", "", song.lyrics.replace('\n', ' '))

In [376]:
get_lyrics("acdc", "back in black")

Searching for songs by acdc...

Changing artist name to 'AC/DC'
Done. Found 0 songs.
Searching for "back in black" by AC/DC...
Done.


"Back in Black Lyrics Back in black, I hit the sack I've been too long, I'm glad to be back Yes, I'm let loose from the noose That's kept me hanging about I'm just looking at the sky 'cause it's getting me high Forget the hearse, 'cause I'll never die I got nine lives, cat's eyes Abusing every one of them and running wild   'Cause I'm back, yes I'm back Well, I'm back, yes I'm back Well, I'm back, back Well, I'm back in black Yes, I'm back in black  Back in the back of a Cadillac Number one with a bullet, I'm a power pack Yes, I'm in a bang with a gang They've got to catch me if they want me to hang 'Cause I'm back on the track and I'm beatin' the flack Nobody's gonna get me on another rap So, look at me now, I'm just making my play Don't try to push your luck, just get out of my way   'Cause I'm back, yes I'm back Well, I'm back, yes I'm back Well, I'm back, back Well, I'm back in black Yes, I'm back in black     Well, I'm back, yes I'm back Well, I'm back, yes I'm back Well, I'm back

In [377]:
def get_df_with_lyrics(spoti_songs_df):
    lyrics_list = []

    for artist, song in zip(spoti_songs_df['artists'], spoti_songs_df['name']):
        try:
            lyrics_list.append(get_lyrics(artist, song))
        except AttributeError:
            lyrics_list.append('None')
    
    spoti_songs_df['lyrics'] = lyrics_list
    return spoti_songs_df

In [ ]:
get_df_with_lyrics(spoti_songs_df_30s)

Searching for songs by Judy Garland...

Done. Found 0 songs.
Searching for "Over The Rainbow" by Judy Garland...
Done.
Searching for songs by Fred Astaire...

Done. Found 0 songs.
Searching for "Night And Day" by Fred Astaire...
Done.
Searching for songs by Glenn Miller...

Done. Found 0 songs.
Searching for "In the Mood" by Glenn Miller...
Specified song does not contain lyrics. Rejecting.
Searching for songs by Fred Astaire...

Done. Found 0 songs.
Searching for "Cheek to Cheek" by Fred Astaire...
Done.
Searching for songs by Ethel Waters...

Done. Found 0 songs.
Searching for "Stormy Weather (Keeps Rainin' All The Time) - 78rpm Version" by Ethel Waters...
No results found for: 'Stormy Weather (Keeps Rainin' All The Time) - 78rpm Version Ethel Waters'
Searching for songs by Artie Shaw...

Done. Found 0 songs.
Searching for "Begin the Beguine" by Artie Shaw...
Done.
Searching for songs by Fred Astaire...

Done. Found 0 songs.
Searching for "The Way You Look Tonight" by Fred Astaire...